# 从零开始的attention

In [3]:
# 引入需要的库和函数
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np
from copy import deepcopy

## n层的克隆

gpt 编码器和解码器需要叠加12层


In [4]:
def clone_module_to_modulelist(module, module_num):
    """
    克隆n个Module类放入ModuleList中，并返回ModuleList，这个ModuleList中的每个Module都是一模一样的
    nn.ModuleList，它是一个储存不同 module，并自动将每个 module 的 parameters 添加到网络之中的容器。
    你可以把任意 nn.Module 的子类 (比如 nn.Conv2d, nn.Linear 之类的) 加到这个 list 里面，
    加入到 nn.ModuleList 里面的 module 是会自动注册到整个网络上的，
    同时 module 的 parameters 也会自动添加到整个网络中。
    :param module: 被克隆的module
    :param module_num: 被克隆的module数
    :return: 装有module_num个相同module的ModuleList
    """
    return nn.ModuleList([deepcopy(module) for _ in range(module_num)])

## 层归一化

In [5]:
class LayerNorm(nn.Module):
    """
    :param x_size: 特征的维度
    :param eps: eps是一个平滑的过程，取值通常在（10^-4~10^-8 之间）
    其含义是，对于每个参数，随着其更新的总距离增多，其学习速率也随之变慢。
    防止出现除以0的情况。

    nn.Parameter将一个不可训练的类型Tensor转换成可以训练的类型parameter，
    并将这个parameter绑定到这个module里面。
    使用这个函数的目的也是想让某些变量在学习的过程中不断的修改其值以达到最优化。
    """
    def __init__(self, x_size, eps = 1e-6):
        super().__init__()
        self.x_size = x_size
        self.eps = eps
        self.alapha = nn.Parameter(torch.ones(self.x_size))
        self.bias = nn.Parameter(torch.zeros(self.x_size))
    
    def forward(self, X):
        # keepdim=True 是一个关键参数，其作用是保持结果张量的维度数量与输入张量 X 相同。
        mean = X.mean(-1, keepdim=True) # 求均值
        std = X.std(-1, keepdim=True)  # 求标准差
        norm = self.alpha * (X - mean) / (std + self.eps) + self.bias
        return norm

## 词向量编码

In [6]:
class WordEmbedding(nn.Module):
    """
    把向量构造成d_model维度的词向量，以便后续送入编码器
    """
    def __init__(self, vocab_size, d_model):
        """
        :param vocab_size: 字典长度
        :param d_model: 词向量维度
        """
        super().__init__()
        self.d_model = d_model
        # 字典中有vocab_size个词，词向量维度是d_model，每个词将会被映射成d_model维度的向量
        self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, X):
        return self.embedding(X)

## 位置编码
https://0809zheng.github.io/2022/07/01/posencode.html 

绝对位置编码 ==> 
1. 三角位置编码

     $$PE_{(pos,2i)} = sin(pos / 10000^{2i/d_{\text{model}}})$$

    $$PE_{(pos,2i+1)} = cos(pos / 10000^{2i/d_{\text{model}}})$$   


其中，pos表示单词所在的位置。2i和2i+1表示位置编码向量中的对应维度。d则为总维度

In [9]:
class PositionalEncoding(nn.Module):
    """
    正弦位置编码，即通过三角函数构建位置编码

    Implementation based on "Attention Is All You Need"
    :cite:`DBLP:journals/corr/VaswaniSPUJGKP17`
    """
    def __init__(self, d_model, dropout, max_seq_len = 5000):
        """
        :param d_mode: 位置向量的向量维度，一般与词向量维度相同，即d_model
        :param dropout: Dropout层的比率
        :param max_len: 句子的最大长度
        """
        super().__init__()
        self.drop_out = nn.Dropout(p=dropout)
        self.d_model = d_model
        # 判断能够构建位置向量
        if d_model % 2 != 0:
            raise ValueError(f"不能使用 sin/cos 位置编码，得到了奇数的维度{d_model:d}，应该使用偶数维度")
        
        """
        构建位置编码pe
        pe公式为：
        PE(pos,2i/2i+1) = sin/cos(pos/10000^{2i/d_{model}})
        """
        pe = torch.zeros(max_seq_len, d_model) # 初始化pe
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** (i / d_model)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** (i / d_model)))
        # 在最前面添加一个批次维度，
        # 形状从 [max_seq_len, d_model] 变成 [1, max_seq_len, d_model]
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        # register_buffer: 将 pe 注册为一个不可训练的张量（不会计算梯度）
    def forward(self, X):
        """
        词向量和位置编码拼接并输出
        :param X: 词向量序列（FloatTensor），``(seq_len, batch_size, self.dim)``
        :return: 词向量和位置编码的拼接
        """
        X = X * math.sqrt(self.d_model)
        seq_len = X.size(1)
        X = X + self.pe[:, :seq_len]
        X = self.drop_out(X)
        return X

### **1. 为什么需要 `X * math.sqrt(self.d_model)`？**

#### **背景**
Transformer 模型中的输入 `X` 通常是通过嵌入层（embedding layer）生成的，维度是 `[batch_size, seq_len, d_model]`。这些嵌入的值通常是从随机初始化开始训练的，数值范围相对较小。

#### **作用**
1. **数值稳定性**：
   - 嵌入层生成的 `X` 数值较小，而位置编码（`pe`）的值范围可能更大（例如，包含 `sin` 和 `cos` 函数的结果）。
   - 通过乘以 `math.sqrt(d_model)`，让 `X` 和 `pe` 的数值范围更匹配，避免加法时信息失衡。

2. **缩放输入**：
   - 缩放可以防止在模型训练初期，注意力分数（dot product attention）过小或过大，从而影响梯度流。

#### **如果不做会怎样？**
- 如果不做 `X * math.sqrt(d_model)`，数值范围失衡可能导致位置编码对输入信号的干扰过大，影响训练效果。
- 但在很多实现中，也可以省略这一步，实际效果因具体任务而异。

---

### **2. 这一步是否需要 Dropout？如果需要，放在哪里？**

#### **是否需要 Dropout**
- **需要 Dropout** 的场景：
  - 如果位置编码的加法操作直接影响模型的学习稳定性，可以在加入位置编码后进行 Dropout，以缓解过拟合。
- **不需要 Dropout** 的场景：
  - 如果模型有其他地方的正则化（如嵌入层、注意力层或前馈网络中使用 Dropout），可以不在这里再添加 Dropout。

#### **Dropout 的位置**
如果需要 Dropout，应该放在位置编码加法之后：
```python
X = X + self.pe[:, :seq_len]
X = self.dropout(X)
```
这是因为位置编码只需要加入一次，Dropout 应该作用在最终的结合结果上。

#### **为什么可能不需要 Dropout**
- 位置编码本身是一个固定的常量，并不会参与训练。如果模型的主要学习部分（如注意力层和前馈网络）已经加了足够的 Dropout，这里再加可能是多余的。

## 自注意力计算

In [13]:
def self_attention(query, key, value, dropout = None, mask = None):
    """
    自注意力计算
    :param query: Q
    :param key: K
    :param value: V
    :param dropout: drop比率
    :param mask: 是否mask
    :return: 经自注意力机制计算后的值
    """
    d_k = key.size(-1) # 防止softmax未来求梯度消失时的d_k
    # Q,K相似度计算公式：\frac{Q^TK}{\sqrt{d_k}}
    attention_weight = query @ key.transpose(-1, -2) / math.sqrt(d_k)
    if mask is not None:
        """
        attention_weight.masked_fill默认是按照传入的mask中为1的元素所在的索引，
        在attention_weight中相同的的索引处替换为value，替换值为负无穷，使得softmax中计算直接让其为0
        """
        # mask.cuda()
        # 进行mask操作，由于参数mask==0，因此替换上述mask中为0的元素所在的索引
        attention_weight = attention_weight.masked_fill(mask == 0, float('-inf'))
    # (batch_size, seq_q, seq_k)
    # Q1 ==> k1, k2, k3...
    # K1 ==> q1, q2, q3...
    attention_weight = F.softmax(attention_weight, dim=-1)  # 进行softmax
    if dropout is not None:
        attention_weight = dropout(attention_weight)
    
    attention_value = attention_weight @ value
    return attention_value, attention_weight 

## 多头注意力

In [ ]:
class MutiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout):
        super().__init__()
        """
        :param head: 头数
        :param d_model: 词向量的维度，必须是head的整数倍
        :param dropout: drop比率
        """
        assert (d_model % head == 0)  # 确保词向量维度是头数的整数倍
        self.heads = heads
        self.d_model = d_model
        self.head_dim = self.d_model // self.heads
        """
        由于多头注意力机制是针对多组Q、K、V，因此有了下面这四行代码，具体作用是，
        针对未来每一次输入的Q、K、V，都给予参数进行构建
        其中linear_out是针对多头汇总时给予的参数
        """
        self.Q_proj = nn.Linear(d_model, d_model)  # 进行一个普通的全连接层变化，但不修改维度
        self.K_proj = nn.Linear(d_model, d_model)
        self.V_proj = nn.Linear(d_model, d_model)
        self.OUT_proj = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(p=dropout)
    def forward(self, query, key, value, mask=None):
        if mask is not None:
            """
            多头注意力机制的线性变换层是4维，是把query[batch, seq_len, d_model]变成[batch, -1, heads, head_dim]
            再1，2维交换变成[batch, heads, -1, head_dim], 所以mask要在第二维（head维）添加一维，与后面的self_attention计算维度一样
            具体点将，就是：
            因为mask的作用是未来传入self_attention这个函数的时候，作为masked_fill需要mask哪些信息的依据
            针对多head的数据，Q、K、V的形状维度中，只有head是通过view计算出来的，是多余的，为了保证mask和
            view变换之后的Q、K、V的形状一直，mask就得在head这个维度添加一个维度出来，进而做到对正确信息的mask
            """
            mask = mask.unsqueeze(1)

        batch_size = query.size(0)  # batch_size大小，假设query的维度是：[10, 32, 512]，其中10是batch_size的大小
        """
        下列三行代码都在做类似的事情，对Q、K、V三个矩阵做处理
        其中view函数是对Linear层的输出做一个形状的重构，其中-1是自适应（自主计算）
        从这种重构中，可以看出，虽然增加了头数，但是数据的总维度是没有变化的，也就是说多头是对数据内部进行了一次拆分
        transopose(1,2)是对前形状的两个维度(索引从0开始)做一个交换，例如(2,3,4,5)会变成(2,4,3,5)
        因此通过transpose可以让view的第二维度参数变成n_head
        假设Linear成的输出维度是：[10, 32, 512]，其中10是batch_size的大小
        注：这里解释了为什么d_model // head == head_dim，如若不是，则view函数做形状重构的时候会出现异常
        """
        query = self.Q_proj(query).view(batch_size, -1, self.head, self.head_dim).transpose(1, 2)  # [b, 8, 32, 64]，head=8
        key = self.K_proj(key).view(batch_size, -1, self.head, self.head_dim).transpose(1, 2)  # [b, 8, 28, 64]
        value = self.V_proj(value).view(batch_size, -1, self.head, self.head_dim).transpose(1, 2)  # [b, 8, 28, 64]
        
        # x是通过自注意力机制计算出来的值， self.attn_softmax是相似概率分布
        X, self.attn_softmax = self_attention(query, key, value, dropout=self.dropout, mask=mask)

        """
        下面的代码是汇总各个头的信息，拼接后形成一个新的x
        其中self.head * self.head_dim，可以看出x的形状是按照head数拼接成了一个大矩阵，然后输入到linear_out层添加参数
        contiguous()是重新开辟一块内存后存储x，然后才可以使用.view方法，否则直接使用.view方法会报错
        """
        X = X.transpose(1, 2).contiguous().view(n_batch, -1, self.head * self.head_dim)
        return self.OUT_proj(X)


## 前馈神经网络FNN
主要是：接收注意力层的输入，并进行升高维度 - 激活函数 - 减低维度

除了attention子层之外，我们的编码器和解码器中的每个层都包含一个全连接的前馈网络，该网络在每个层的位置相同（都在每个encoder-layer或者decoder-layer的最后）。该前馈网络包括两个线性变换，并在两个线性变换中间有一个ReLU激活函数。

$$\mathrm{FFN}(x)=\max(0, xW_1 + b_1) W_2 + b_2$$                                                                        

尽管两层都是线性变换，但它们在层与层之间使用不同的参数。另一种描述方式是两个内核大小为1的卷积。 输入和输出的维度都是 $d_{\text{model}}=512$, 内层维度是$d_{ff}=2048$。（也就是第一层输入512维,输出2048维；第二层输入2048维，输出512维）

In [16]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout):
        '''
        :param d_model: FFN第一层输入的维度
        :param d_ff: FNN第二层隐藏层输入的维度，一般为4 * d_model
        :param dropout: drop比率
        '''
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff) # 升维度
        self.linear2 = nn.Linear(d_ff, d_model) # 降维度
        self.ff_dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        
    def forward(self, X):
        """
        :param x: 输入数据，形状为(batch_size, input_len, model_dim)
        :return: 输出数据（FloatTensor），形状为(batch_size, input_len, model_dim)
        """
        inter = self.ff_dropout(F.relu(self.linear1(self.layer_norm(X)))) # 升维度加激活
        output = self.linear2(inter)
        return output

## 残差链接

In [15]:
class SublayerConnection(nn.Module):
    """
    子层的连接: layer_norm(x + sublayer(x))
    上述可以理解为一个残差网络加上一个LayerNorm归一化
    """

    def __init__(self, size, dropout=0.1):
        """
        :param size: d_model
        :param dropout: drop比率
        """
        super(SublayerConnection, self).__init__()
        self.layer_norm = LayerNorm(size)
        # TODO：在SublayerConnection中LayerNorm可以换成nn.BatchNorm2d
        # self.layer_norm = nn.BatchNorm2d()
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, sublayer):
        return self.dropout(self.layer_norm(x + sublayer(x)))


## 单一encoder层

In [17]:
class EncoderLayer(nn.Module):
    """
    一层编码Encoder层
    MultiHeadAttention -> Add & Norm -> Feed Forward -> Add & Norm
    """

    def __init__(self, size, attn, feed_forward, dropout=0.1):
        """
        :param size: d_model
        :param attn: 已经初始化的Multi-Head Attention层
        :param feed_forward: 已经初始化的Feed Forward层
        :param dropout: drop比率
        """
        super(EncoderLayer, self).__init__()
        self.attn = attn
        self.feed_forward = feed_forward

        """
        下面一行的作用是因为一个Encoder层具有两个残差结构的网络
        因此构建一个ModuleList存储两个SublayerConnection，以便未来对数据进行残差处理
        """
        self.sublayer_connection_list = clone_module_to_modulelist(SublayerConnection(size, dropout), 2)

    def forward(self, x, mask):
        """
        :param x: Encoder层的输入
        :param mask: mask标志
        :return: 经过一个Encoder层处理后的输出
        """
        """
        编码层第一层子层
        self.attn 应该是一个已经初始化的Multi-Head Attention层
        把Encoder的输入数据x和经过一个Multi-Head Attention处理后的x_attn送入第一个残差网络进行处理得到first_x
        """
        first_x = self.sublayer_connection_list[0](x, lambda x_attn: self.attn(x, x, x, mask))

        """
        编码层第二层子层
        把经过第一层子层处理后的数据first_x与前馈神经网络送入第二个残差网络进行处理得到Encoder层的输出
        """
        return self.sublayer_connection_list[1](first_x, self.feed_forward)

## 单一decoder层


In [18]:
class DecoderLayer(nn.Module):
    """
    一层解码Decoder层
    Mask MultiHeadAttention -> Add & Norm -> Multi-Head Attention -> Add & Norm
    -> Feed Forward -> Add & Norm
    """

    def __init__(self, d_model, attn, feed_forward, sublayer_num, dropout=0.1):
        """
        :param d_model: d_model
        :param attn: 已经初始化的Multi-Head Attention层
        :param feed_forward: 已经初始化的Feed Forward层
        :param sublayer_num: 解码器内部子层数，如果未来r2l_memory传入有值，则为4层，否则为普通的3层
        :param dropout: drop比率
        """
        super(DecoderLayer, self).__init__()
        self.attn = attn
        self.feed_forward = feed_forward
        self.sublayer_connection_list = clone_module_to_modulelist(SublayerConnection(d_model, dropout), sublayer_num)

    def forward(self, x, l2r_memory, src_mask, trg_mask, r2l_memory=None, r2l_trg_mask=None):
        """
        :param x: Decoder的输入(captioning)
        :param l2r_memory: Encoder的输出，作为Multi-Head Attention的K，V值，为从左到右的Encoder的输出
        :param src_mask: 编码器输入的填充掩码
        :param trg_mask: 解码器输入的填充掩码和序列掩码，即对后面单词的掩码
        :param r2l_memory: 从右到左解码器的输出
        :param r2l_trg_mask: 从右到左解码器的输出的填充掩码和序列掩码
        :return: Encoder的输出
        """
        """
        解码器第一层子层
        把Decoder的输入数据x和经过一个Masked Multi-Head Attention处理后的first_x_attn送入第一个残差网络进行处理得到first_x
        """
        first_x = self.sublayer_connection_list[0](x, lambda first_x_attn: self.attn(x, x, x, trg_mask))

        """
        解码器第二层子层
        把第一层子层得到的first_x和
        经过一个Multi-Head Attention处理后的second_x_attn（由first_x和Encoder的输出进行自注意力计算）
        送入第二个残差网络进行处理
        """
        second_x = self.sublayer_connection_list[1](first_x,
                                                    lambda second_x_attn: self.attn(first_x, l2r_memory, l2r_memory,
                                                                                    src_mask))

        """
        解码器第三层子层
        把经过第二层子层处理后的数据second_x与前馈神经网络送入第三个残差网络进行处理得到Decoder层的输出
        
        如果有r2l_memory数据，则还需要经过一层多头注意力计算，也就是说会有四个残差网络
        r2l_memory是让Decoder层多了一层双向编码中从右到左的编码层
        而只要三个残差网络的Decoder层只有从左到右的编码
        """
        if not r2l_memory:
            # 进行从右到左的编码，增加语义信息
            third_x = self.sublayer_connection_list[-2](second_x,
                                                        lambda third_x_attn: self.attn(second_x, r2l_memory, r2l_memory,
                                                                                       r2l_trg_mask))
            return self.sublayer_connection_list[-1](third_x, self.feed_forward)
        else:
            return self.sublayer_connection_list[-1](second_x, self.feed_forward)